In [1]:
# Python Notebook for PHYS 422 Assignment #3

import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
from matplotlib import rc
import scipy as sp

# extra stuff to make the plots look nice:

from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

params = {"text.usetex" : True,
          "font.family" : "serif",
          "font.serif" : ["cmr10"],
          "font.size": 20
         }
plt.rcParams.update(params)
plt.rcParams['axes.formatter.use_mathtext'] = True

import os
os.environ["PATH"]
os.environ["PATH"] += os.pathsep + '/Library/TeX/texbin/'
plt.rcParams['axes.edgecolor'] = 'black'
plt.rcParams['xtick.color'] = 'black'
plt.rcParams['ytick.color'] = 'black'

# Set tick parameters using rcParams
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['xtick.major.size'] = 8
plt.rcParams['xtick.minor.size'] = 4
plt.rcParams['ytick.major.size'] = 8
plt.rcParams['ytick.minor.size'] = 4
plt.rcParams['xtick.major.width'] = 0.7
plt.rcParams['xtick.minor.width'] = 0.7
plt.rcParams['ytick.major.width'] = 0.7
plt.rcParams['ytick.minor.width'] = 0.7

plt.rcParams['font.weight'] = 'bold'

from matplotlib.colors import PowerNorm


## b. [Computational]
The above may have caused you to think of the expression from electrostatics for the force on an electric dipole: 

$$
\vec{F} = (\vec{p} \cdot \nabla)\vec{E} = (\alpha\vec{E} \cdot \nabla)\vec{E}
$$

where as here from electrodynamics in this
situation we have

$$
\vec{F} = \frac{\alpha}{2} \nabla E^2
$$

Here consider the electric field specified in Question 2 and create a program to trace the trajectory of a polarizable particle in the field that starts at a point away from the origin, using both the equation from electrostatics and the one from the above (ignore the cross term). 

You only need simulate the motion of the particle over 5 complete cycles. The particle starts from rest and other forces such as from gravity, radiation pressure, and drag from any surrounding fluid can be ignored for this question. 

For some realism consider a Caesium atom (see Table 4.1 for polarizability), in a 500 nm wavelength laser illumination, where the Cs atom starts at a position 0.5 mm from the origin in the z = 0 plane, and the peak electric field strength at that point is 104 V/m. Plot and compare the resulting trajectories in a format that you think best shows what is happening. 



The equations governing motion here for the particle can be written as:

$$
\vec{F}=ma=m\ddot r = \alpha \left(\frac{1}{2} \nabla E^2 +\frac{\partial}{\partial t}(\vec{E} \times \vec{B})\right)
$$

For the electrodynamical case, and:

$$
\vec{F}=ma=m\ddot r = (\alpha \vec{E} \cdot  \nabla)\vec{E}
$$

In the electrostatic case. 

We will ignore the cross product term in the first equation for simplicity. Our field is governed by the expression found in problem 2:

$$
\vec{E} = -\frac{A sin \theta}{r}\left( sin u + \frac{1}{kr} cos u \right) \hat{\phi}
$$

with $u = kr - wt$. Our problem is as follows:

1. Convert the above equation to something that we may visualize in cartesian using the spherical coordinate identities.
2. Program a function that defines our field, including all constants and vector components.
3. Plug the field into the expression for force.
4. Numerically solve the differential equation to obtain the position vector.
5. Plot the position vector $\vec{r}(t)$.

From Griffith's we know that:

$$
r = \sqrt{x^2 +y^2 +z^2}
$$

$$
\hat{\phi} = -sin \phi \ \hat{x} +cos\phi \ \hat{y}
$$

Hmmmmm it might be easier to do this in spherical and then convert at the end. The gradient in spherical coordinates is (not our E yet):

$$
\nabla E = \frac{\partial E}{\partial r} \hat{r} + \frac{1}{r}\frac{\partial E}{\partial \theta} \hat{\theta} + \frac{1}{rsin \theta} \frac{\partial E}{\partial \phi} \hat{\phi}
$$

Divergence in spherical coordinates is given by:

$$
\nabla \cdot \mathbf{E} = \frac{1}{r^2} \frac{\partial}{\partial r} (r^2 E_r) + \frac{1}{r \sin \theta} \frac{\partial}{\partial \theta} (\sin \theta E_\theta) + \frac{1}{r \sin \theta} \frac{\partial E_\phi}{\partial \phi}
$$

In [6]:
# let's first get our bearings by defining all pertinent constants:

epsilon_0 = 8.854187817e-12
pi = np.pi
c = 299792458 # m/s

# polarizability of cesium:

alp = 59.4 *10**(-30) * 4* np.pi * epsilon_0 # m^3

# Going to just interpret the constant A as the given peak electric field strength:

A = 10**(4) # V/m

# wavelength, frequency and wavenumber of the light:

lamb = 500 * 10**(-9); k = (2*pi)/lamb
w = (2*pi*c)/lamb

# starting position/velocity:

r_0c = (0.0005, 0, 0) # starting only in x
r_0s = (0.0005, 0, 0)
v_0c = (0,0,0)

In [7]:
# first step of the process is to create a function for the field:

def electric_field(r, theta, t,  k=k, A=A, w=w):
    
    u = k*r - w*t
    
    E_phi = - ((A*np.sin(theta))/r) *(np.sin(u) + (1/k*r) * np.cos(u))
    
    return (0, 0, E_phi)

# next step is to make a simple squared E function:

def E_sq(r, theta, t,  k=k, A=A, w=w):
    
    E = electric_field(r, theta, t,  k=k, A=A, w=w)
    
    E_squared = sum([i**2 for i in E])
    
    return E_squared

# we now want to create a gradient function on E^2:

def E_grad(r, theta, t,  k=k, A=A, w=w):
    
    r_grad = np.gradient(E_sq(r, theta, t,  k=k, A=A, w=w), r)
    
    theta_grad = np.gradient(E_sq(r, theta, t,  k=k, A=A, w=w), theta)/r
    
    phi_grad = 0 # by inspection
    
    return (r_grad, theta_grad, phi_grad)

# finally we want to integrate with respect to t twice:


Now at this point in the analysis, we may either integrate each component twice to get the force or using the some kind of solver method like Euler or what not to continuously update the $\vec{r}$ and get the path of motion. This would be in spherical coordinates which we could then convert to cartesian and plot. 

Further, to get the electrostatics case we would take the spherical divergence of the field which is fairly straightforward using the above equation and then multiplying that by the field once more. The using solver method or integrating we could get the path of motion. When we look at the form of the divergence and the dependence of E on $\phi$ it seems as though the force  might equal to zero, which definitely would not be the case in the electrostatics case.

The debugging of this problem would take a considerable amount of time that I currently do not have access to. If time permits in my life I will revisit this problem but it seems as though this might be where it ends up...